# One-sided Wilcoxon SIGNED test - bias
By: Sam<br>
Updated at: 27/04/2023<br>
Compare performance of discretizers, ONE-SIDED (greater) <br>

===

Matched pairs settings
- Sample: 540 ML models after discretization (as at 27/04/2023)
- Purpose: pair-wise comparison metrics of the models using different discretization method
    - Test 1.1: Compare bias between pairs of discretizers, regardless of algorithms
    - Test 1.2-1.6: Compare bias between pairs of discretizers, filter by models (CNB, ID3, Knn-Hamming, Knn-VDM, KNN)
    
===

Input data: instrinsic properties and model performance metrics <br>
ChiMerge manuallly and ChiMerge-SB are merged together. <br>
!!! **NB: Please update the data for metrics and export to csv before running this script!

In [1]:
# Import library
import pandas as pd
import numpy as np
from scipy import stats
import math
import random

In [2]:
# Import evaluation data (updated at 27/04/2023)
data = pd.read_csv("all_evaluation_270423.csv")

In [3]:
data.head()

,dataset,disc,param,inconsistency,models,con_features,size,time_disc,accuracy,time_train,bias,variance
0,iris,EWD,4,0.0667,ID3,4,150,0.0164,0.84,0.008698225,0.158,0.055
1,iris,EWD,7,0.02,ID3,4,150,0.0157,0.79,0.010634899,0.158,0.054
2,iris,EWD,10,0.0067,ID3,4,150,0.0164,0.95,0.010643005,0.053,0.014
3,iris,EFD,4,0.04,ID3,4,150,0.0167,0.84,0.009439945,0.158,0.049
4,iris,EFD,7,0.04,ID3,4,150,0.0239,0.95,0.010675907,0.053,0.07


In [4]:
data['disc'].unique() # get list of discretizers

array(['EWD', 'EFD', 'FFD', 'ChiMerge', 'DT', 'ChiMerge-SB'], dtype=object)

In [5]:
data.columns

Index(['dataset', 'disc', 'param', ' inconsistency ', 'models', 'con_features',
       'size', 'time_disc', 'accuracy', 'time_train', 'bias', 'variance'],
      dtype='object')

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 540 entries, 0 to 539
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   dataset          540 non-null    object 
 1   disc             540 non-null    object 
 2   param            540 non-null    int64  
 3    inconsistency   540 non-null    object 
 4   models           540 non-null    object 
 5   con_features     540 non-null    int64  
 6   size             540 non-null    object 
 7   time_disc        540 non-null    float64
 8   accuracy         540 non-null    object 
 9   time_train       540 non-null    object 
 10  bias             540 non-null    object 
 11  variance         540 non-null    object 
dtypes: float64(1), int64(2), object(9)
memory usage: 50.8+ KB


In [7]:
data['models'].unique()

array(['ID3', 'CNB', 'Knn-VDM', 'Knn-Hamming'], dtype=object)

# Wilcoxon_Complete pipeline

## 1.  Wilcoxon signed t test, bias
Ref: https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.wilcoxon.html
Implement 4 replications: (DONE)
- Regardless algorithms
- Filter for each algorithm: CNB, ID3, Knn

## 1.1 bias, no filter in algorithm

In [8]:
# Preparation: Prepare list of metrics for each discretization (test_list)
# Step 1: Obtain bias for each discretization, convert into numeric, string values will be return as NaN
ewd_acc = pd.to_numeric(data[data['disc']=="EWD"]['bias'],errors='coerce').tolist()
efd_acc = pd.to_numeric(data[data['disc']=="EFD"]['bias'],errors='coerce').tolist()
ffd_acc = pd.to_numeric(data[data['disc']=="FFD"]['bias'],errors='coerce').tolist()
cm_acc = pd.to_numeric(data[(data['disc']=="ChiMerge") | (data['disc'] == "ChiMerge-SB")]['bias'],errors='coerce').tolist()
dt_acc = pd.to_numeric(data[data['disc']=="DT"]['bias'],errors='coerce').tolist()
# Check number of metrics available for each discretizer
print(len(ewd_acc))
print(len(efd_acc))
print(len(ffd_acc))
print(len(cm_acc))
print(len(dt_acc))

# Step 2: filter numeric values
raw_list = [ewd_acc, efd_acc, ffd_acc, cm_acc, dt_acc]
num_list = [] # filter numeric values only
for metric in raw_list:
    metric_new = [x for x in metric if (math.isnan(x) == False)] # using list comprehension
    num_list.append(metric_new)
    
# Step 3: random sample
# For discretization methods with different value, randomly select so that the final sample size are equal
# Reference: https://docs.python.org/3/library/random.html
k = min(len(metric) for metric in num_list)
test_list = []
for metric in num_list:
    random.seed(20)
    if len(metric) > k:
        metric = random.sample(metric, k=k)
    else: metric = metric
    #print(metric)
    test_list.append(metric)

90
90
120
120
120


In [9]:
# data[(data['disc']=="ChiMerge") | (data['disc'] == "ChiMerge-BS")]['bias']

In [10]:
# Initialiation
disc_key = ['ewd', 'efd', 'ffd', 'cm', 'dt']
disc_value = test_list # list of metrics for each discretization after preparation
test_stat = []
p_value = []
disc_compare = []
# Create dictionary store discretization and series of bias
disc = {}
for key in disc_key:
    for value in disc_value:
        disc[key] = value
        disc_value.remove(value)
        break  

# Create loop for Wilcoxon test (two sided) - handle diff = 0
for i in disc_key:
    for j in disc_key:
        if i != j:
            disc_compare.append(f'{i} vs {j}')
            # print(f'{i} vs {j}')
            # Compute difference list
            diff_list = []
            for m in range(0, len(disc[i])):
                diff = disc[i][m] - disc[j][m]
                diff_list.append(diff)
                                
            if all(item == 0 for item in diff_list) == False: # if the diff list does not contain all 0
                test_stat.append(stats.wilcoxon(diff_list, alternative = 'greater').statistic)
                p_value.append(stats.wilcoxon(diff_list, alternative = 'greater').pvalue)
            else: # if the diff list contain only0, cannot do Wilcoxon test
                error = 'N/A'
                test_stat.append(error)
                p_value.append(error)


In [11]:
# Convert 3 lists to dataframe
disc_compare = pd.DataFrame(disc_compare, columns=['disc_compare'])
test_stat = pd.DataFrame(test_stat, columns=['wtest_stat'])
p_value = pd.DataFrame(p_value, columns=['p_value'])

In [12]:
# Result table
wt_result = pd.concat([disc_compare, test_stat, p_value], axis = 1)
# Drop duplicate
wt_result.drop_duplicates(subset=['p_value'], inplace=True)

In [13]:
wt_result

,disc_compare,wtest_stat,p_value
0,ewd vs efd,1925.5,0.045659
1,ewd vs ffd,1909.0,0.053932
2,ewd vs cm,1918.0,0.030038
3,ewd vs dt,2060.0,0.009493
4,efd vs ewd,1234.5,0.954341
5,efd vs ffd,1514.5,0.473686
6,efd vs cm,1723.0,0.181677
7,efd vs dt,1797.5,0.100261
8,ffd vs ewd,1251.0,0.946068
9,ffd vs efd,1488.5,0.526314


In [14]:
# Exporting result

model = 'all'
metric = 'bias'
test = 'one_sided'

filename = f"wilcoxon_{metric}_{model}-{test}.csv"
wt_result['model'] = model
wt_result['metric'] = metric

wt_result.to_csv(filename, index = False)

## 1.2 bias, only CNB

(data['disc']=="ChiMerge") | (data['disc'] == "ChiMerge-SB")

In [15]:
# Preparation: Prepare list of metrics for each discretization (test_list)
# Step 1: Obtain bias for each discretization, convert into numeric, string values will be return as NaN
# Filter for CNB
# Obtain bias for each discretization, convert into numeric, string values will be return as NaN
ewd_acc_cnb = pd.to_numeric(data[(data['disc']=="EWD") & (data['models']=="CNB")]['bias'],errors='coerce').tolist()
efd_acc_cnb = pd.to_numeric(data[(data['disc']=="EFD") & (data['models']=="CNB")]['bias'],errors='coerce').tolist()
ffd_acc_cnb = pd.to_numeric(data[(data['disc']=="FFD") & (data['models']=="CNB")]['bias'],errors='coerce').tolist()
cm_acc_cnb = pd.to_numeric(data[((data['disc']=="ChiMerge") | (data['disc'] == "ChiMerge-SB")) & (data['models']=="CNB")]['bias'],errors='coerce').tolist()
dt_acc_cnb = pd.to_numeric(data[(data['disc']=="DT") & (data['models']=="CNB")]['bias'],errors='coerce').tolist()
# Check number of metrics available for each discretizer
print(len(ewd_acc_cnb))
print(len(efd_acc_cnb))
print(len(ffd_acc_cnb))
print(len(cm_acc_cnb))
print(len(dt_acc_cnb))

# Step 2: filter numeric values
raw_list = [ewd_acc_cnb,efd_acc_cnb, ffd_acc_cnb, cm_acc_cnb, dt_acc_cnb]
num_list = [] # filter numeric values only
for metric in raw_list:
    metric_new = [x for x in metric if (math.isnan(x) == False)] # using list comprehension
    num_list.append(metric_new)
    
# Step 3: random sample
# For discretization methods with different value, randomly select so that the final sample size are equal
# Reference: https://docs.python.org/3/library/random.html
k = min(len(metric) for metric in num_list)
test_list = []
for metric in num_list:
    random.seed(20)
    if len(metric) > k:
        metric = random.sample(metric, k=k)
    else: metric = metric
    #print(metric)
    test_list.append(metric)

30
30
40
40
40


In [16]:
# Initialiation
disc_key = ['ewd', 'efd', 'ffd', 'cm', 'dt']
disc_value = test_list # list of metrics for each discretization after preparation
test_stat = []
p_value = []
disc_compare = []
# Create dictionary store discretization and series of bias
disc = {}
for key in disc_key:
    for value in disc_value:
        disc[key] = value
        disc_value.remove(value)
        break  

# Create loop for Wilcoxon test (two sided) - handle diff = 0
for i in disc_key:
    for j in disc_key:
        if i != j:
            disc_compare.append(f'{i} vs {j}')
            # print(f'{i} vs {j}')
            # Compute difference list
            diff_list = []
            for m in range(0, len(disc[i])):
                diff = disc[i][m] - disc[j][m]
                diff_list.append(diff)
                # print(diff_list)
                
            if all(item == 0 for item in diff_list) == False: # if the diff list does not contain all 0
                test_stat.append(stats.wilcoxon(diff_list, alternative = 'greater').statistic)
                p_value.append(stats.wilcoxon(diff_list, alternative = 'greater').pvalue)
            else: # if the diff list contain only0, cannot do Wilcoxon test
                error = 'N/A'
                test_stat.append(error)
                p_value.append(error)

/Users/anhtth/opt/anaconda3/lib/python3.9/site-packages/scipy/stats/morestats.py:3141: UserWarning: Exact p-value calculation does not work if there are ties. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "


In [17]:
# Convert 3 lists to dataframe
disc_compare = pd.DataFrame(disc_compare, columns=['disc_compare'])
test_stat = pd.DataFrame(test_stat, columns=['wtest_stat'])
p_value = pd.DataFrame(p_value, columns=['p_value'])

In [18]:
# Result table
wt_result = pd.concat([disc_compare, test_stat, p_value], axis = 1)
# Drop duplicate
wt_result.drop_duplicates(subset=['p_value'], inplace=True)

In [19]:
wt_result

,disc_compare,wtest_stat,p_value
0,ewd vs efd,137.0,0.227444
1,ewd vs ffd,169.0,0.088096
2,ewd vs cm,171.5,0.077985
4,efd vs ewd,94.0,0.772556
5,efd vs ffd,115.0,0.648808
6,efd vs cm,136.0,0.387264
7,efd vs dt,138.0,0.363092
8,ffd vs ewd,84.0,0.917071
10,ffd vs cm,148.0,0.251420
11,ffd vs dt,134.5,0.254452


In [20]:
# Exporting result

model = 'CNB'
metric = 'bias'
test = 'one_sided'

filename = f"wilcoxon_{metric}_{model}-{test}.csv"
wt_result['model'] = model
wt_result['metric'] = metric

wt_result.to_csv(filename, index = False)

## 1.3 bias, only ID3

In [21]:
# Preparation: Prepare list of metrics for each discretization (test_list)
# Step 1: Obtain bias for each discretization, convert into numeric, string values will be return as NaN
# Filter for ID3
# Obtain bias for each discretization, convert into numeric, string values will be return as NaN
ewd_acc_ID3 = pd.to_numeric(data[(data['disc']=="EWD") & (data['models']=="ID3")]['bias'],errors='coerce').tolist()
efd_acc_ID3 = pd.to_numeric(data[(data['disc']=="EFD") & (data['models']=="ID3")]['bias'],errors='coerce').tolist()
ffd_acc_ID3 = pd.to_numeric(data[(data['disc']=="FFD") & (data['models']=="ID3")]['bias'],errors='coerce').tolist()
cm_acc_ID3 = pd.to_numeric(data[((data['disc']=="ChiMerge") | (data['disc'] == "ChiMerge-SB")) & (data['models']=="ID3")]['bias'],errors='coerce').tolist()
dt_acc_ID3 = pd.to_numeric(data[(data['disc']=="DT") & (data['models']=="ID3")]['bias'],errors='coerce').tolist()
# Check number of metrics available for each discretizer
print(len(ewd_acc_ID3))
print(len(efd_acc_ID3))
print(len(ffd_acc_ID3))
print(len(cm_acc_ID3))
print(len(dt_acc_ID3))

# Step 2: filter numeric values
raw_list = [ewd_acc_ID3,efd_acc_ID3, ffd_acc_ID3, cm_acc_ID3, dt_acc_ID3]
num_list = [] # filter numeric values only
for metric in raw_list:
    metric_new = [x for x in metric if (math.isnan(x) == False)] # using list comprehension
    num_list.append(metric_new)
    
# Step 3: random sample
# For discretization methods with different value, randomly select so that the final sample size are equal
# Reference: https://docs.python.org/3/library/random.html
k = min(len(metric) for metric in num_list)
test_list = []
for metric in num_list:
    random.seed(20)
    if len(metric) > k:
        metric = random.sample(metric, k=k)
    else: metric = metric
    #print(metric)
    test_list.append(metric)

30
30
40
40
40


In [22]:
# Initialiation
disc_key = ['ewd', 'efd', 'ffd', 'cm', 'dt']
disc_value = test_list # list of metrics for each discretization after preparation
test_stat = []
p_value = []
disc_compare = []
# Create dictionary store discretization and series of bias
disc = {}
for key in disc_key:
    for value in disc_value:
        disc[key] = value
        disc_value.remove(value)
        break  

# Create loop for Wilcoxon test (two sided) - handle diff = 0
for i in disc_key:
    for j in disc_key:
        if i != j:
            disc_compare.append(f'{i} vs {j}')
            # print(f'{i} vs {j}')
            # Compute difference list
            diff_list = []
            for m in range(0, len(disc[i])):
                diff = disc[i][m] - disc[j][m]
                diff_list.append(diff)
                                
            if all(item == 0 for item in diff_list) == False: # if the diff list does not contain all 0
                test_stat.append(stats.wilcoxon(diff_list, alternative = 'greater').statistic)
                p_value.append(stats.wilcoxon(diff_list, alternative = 'greater').pvalue)
            else: # if the diff list contain only0, cannot do Wilcoxon test
                error = 'N/A'
                test_stat.append(error)
                p_value.append(error)

In [23]:
# Convert 3 lists to dataframe
disc_compare = pd.DataFrame(disc_compare, columns=['disc_compare'])
test_stat = pd.DataFrame(test_stat, columns=['wtest_stat'])
p_value = pd.DataFrame(p_value, columns=['p_value'])

In [24]:
# Result table
wt_result = pd.concat([disc_compare, test_stat, p_value], axis = 1)
# Drop duplicate
wt_result.drop_duplicates(subset=['p_value'], inplace=True)

In [25]:
wt_result

,disc_compare,wtest_stat,p_value
0,ewd vs efd,270.0,0.008193
1,ewd vs ffd,278.0,0.174666
2,ewd vs cm,257.5,0.303544
3,ewd vs dt,304.0,0.070689
4,efd vs ewd,81.0,0.991807
5,efd vs ffd,224.5,0.565351
6,efd vs cm,197.5,0.764216
7,efd vs dt,243.0,0.414507
8,ffd vs ewd,187.0,0.825334
9,ffd vs efd,240.5,0.434649


In [26]:
# Exporting result

model = 'ID3'
metric = 'bias'
test = 'one_sided'

filename = f"wilcoxon_{metric}_{model}-{test}.csv"
wt_result['model'] = model
wt_result['metric'] = metric

wt_result.to_csv(filename, index = False)

## 1.4 bias, only KNN-VDM


In [27]:
# Preparation: Prepare list of metrics for each discretization (test_list)
# Step 1: Obtain bias for each discretization, convert into numeric, string values will be return as NaN
# Filter for Knn-VDM
# Obtain bias for each discretization, convert into numeric, string values will be return as NaN
ewd_acc_knn_vdm = pd.to_numeric(data[(data['disc']=="EWD") & (data['models']=="Knn-VDM")]['bias'],errors='coerce').tolist()
efd_acc_knn_vdm = pd.to_numeric(data[(data['disc']=="EFD") & (data['models']=="Knn-VDM")]['bias'],errors='coerce').tolist()
ffd_acc_knn_vdm = pd.to_numeric(data[(data['disc']=="FFD") & (data['models']=="Knn-VDM")]['bias'],errors='coerce').tolist()
cm_acc_knn_vdm = pd.to_numeric(data[((data['disc']=="ChiMerge") | (data['disc'] == "ChiMerge-SB")) & (data['models']=="Knn-VDM")]['bias'],errors='coerce').tolist()
dt_acc_knn_vdm = pd.to_numeric(data[(data['disc']=="ChiMerge") & (data['models']=="Knn-VDM")]['bias'],errors='coerce').tolist()

# Check number of metrics available for each discretizer
print(len(ewd_acc_knn_vdm))
print(len(efd_acc_knn_vdm))
print(len(ffd_acc_knn_vdm))
print(len(cm_acc_knn_vdm))
print(len(dt_acc_knn_vdm))

# Step 2: filter numeric values
raw_list = [ewd_acc_knn_vdm,efd_acc_knn_vdm, ffd_acc_knn_vdm, cm_acc_knn_vdm, dt_acc_knn_vdm]
num_list = [] # filter numeric values only
for metric in raw_list:
    metric_new = [x for x in metric if (math.isnan(x) == False)] # using list comprehension
    num_list.append(metric_new)
    
# Step 3: random sample
# For discretization methods with different value, randomly select so that the final sample size are equal
# Reference: https://docs.python.org/3/library/random.html
k = min(len(metric) for metric in num_list)
test_list = []
for metric in num_list:
    random.seed(20)
    if len(metric) > k:
        metric = random.sample(metric, k=k)
    else: metric = metric
    #print(metric)
    test_list.append(metric)

9
9
12
8
8


In [28]:
# Initialiation
disc_key = ['ewd', 'efd', 'ffd', 'cm', 'dt']
disc_value = test_list # list of metrics for each discretization after preparation
test_stat = []
p_value = []
disc_compare = []
# Create dictionary store discretization and series of bias
disc = {}
for key in disc_key:
    for value in disc_value:
        disc[key] = value
        disc_value.remove(value)
        break  

In [29]:
# Create loop for Wilcoxon test (two sided)
for i in disc_key:
    for j in disc_key:
        if i != j:
            disc_compare.append(f'{i} vs {j}')
            # print(f'{i} vs {j}')
            # Compute difference list
            diff_list = []
            for m in range(0, len(disc[i])):
                diff = disc[i][m] - disc[j][m]
                diff_list.append(diff)
                # print(diff_list)
                
            if all(item == 0 for item in diff_list) == False: # if the diff list does not contain all 0
                test_stat.append(stats.wilcoxon(diff_list, alternative = 'greater').statistic)
                p_value.append(stats.wilcoxon(diff_list, alternative = 'greater').pvalue)
            else: # if the diff list contain only0, cannot do Wilcoxon test
                error = 'N/A'
                test_stat.append(error)
                p_value.append(error)

/Users/anhtth/opt/anaconda3/lib/python3.9/site-packages/scipy/stats/morestats.py:3141: UserWarning: Exact p-value calculation does not work if there are ties. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/Users/anhtth/opt/anaconda3/lib/python3.9/site-packages/scipy/stats/morestats.py:3155: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")


In [30]:
# Convert 3 lists to dataframe
disc_compare = pd.DataFrame(disc_compare, columns=['disc_compare'])
test_stat = pd.DataFrame(test_stat, columns=['wtest_stat'])
p_value = pd.DataFrame(p_value, columns=['p_value'])

In [31]:
# Result table
wt_result = pd.concat([disc_compare, test_stat, p_value], axis = 1)
# Drop duplicate
wt_result.drop_duplicates(subset=['p_value'], inplace=True)

In [32]:
wt_result

,disc_compare,wtest_stat,p_value
0,ewd vs efd,2.0,0.863339
1,ewd vs ffd,6.0,0.6875
2,ewd vs cm,4.0,0.84375
4,efd vs ewd,8.0,0.136661
6,efd vs cm,7.0,0.232604
8,ffd vs ewd,9.0,0.40625
9,ffd vs efd,8.5,0.5
12,cm vs ewd,11.0,0.21875
13,cm vs efd,3.0,0.767396
14,cm vs ffd,7.0,0.59375


In [33]:
# Exporting result

model = 'KNN-VDM'
metric = 'bias'
test = 'one_sided'

filename = f"wilcoxon_{metric}_{model}-{test}.csv"
wt_result['model'] = model
wt_result['metric'] = metric

wt_result.to_csv(filename, index = False)

## 1.5 bias, only KNN-Hamming


In [34]:
# Preparation: Prepare list of metrics for each discretization (test_list)
# Step 1: Obtain bias for each discretization, convert into numeric, string values will be return as NaN
# Filter for Knn-VDM
# Obtain bias for each discretization, convert into numeric, string values will be return as NaN
ewd_acc_knn_hamming = pd.to_numeric(data[(data['disc']=="EWD") & (data['models']=="Knn-Hamming")]['bias'],errors='coerce').tolist()
efd_acc_knn_hamming = pd.to_numeric(data[(data['disc']=="EFD") & (data['models']=="Knn-Hamming")]['bias'],errors='coerce').tolist()
ffd_acc_knn_hamming = pd.to_numeric(data[(data['disc']=="FFD") & (data['models']=="Knn-Hamming")]['bias'],errors='coerce').tolist()
cm_acc_knn_hamming = pd.to_numeric(data[((data['disc']=="ChiMerge") | (data['disc'] == "ChiMerge-SB")) & (data['models']=="Knn-Hamming")]['bias'],errors='coerce').tolist()
dt_acc_knn_hamming = pd.to_numeric(data[(data['disc']=="ChiMerge") & (data['models']=="Knn-Hamming")]['bias'],errors='coerce').tolist()

# Check number of metrics available for each discretizer
print(len(ewd_acc_knn_hamming))
print(len(efd_acc_knn_hamming))
print(len(ffd_acc_knn_hamming))
print(len(cm_acc_knn_hamming))
print(len(dt_acc_knn_hamming))

# Step 2: filter numeric values
raw_list = [ewd_acc_knn_hamming,efd_acc_knn_hamming, ffd_acc_knn_hamming, cm_acc_knn_hamming, dt_acc_knn_hamming]
num_list = [] # filter numeric values only
for metric in raw_list:
    metric_new = [x for x in metric if (math.isnan(x) == False)] # using list comprehension
    num_list.append(metric_new)
    
# Step 3: random sample
# For discretization methods with different value, randomly select so that the final sample size are equal
# Reference: https://docs.python.org/3/library/random.html
k = min(len(metric) for metric in num_list)
test_list = []
for metric in num_list:
    random.seed(20)
    if len(metric) > k:
        metric = random.sample(metric, k=k)
    else: metric = metric
    #print(metric)
    test_list.append(metric)

21
21
28
32
12


In [35]:
# Initialiation
disc_key = ['ewd', 'efd', 'ffd', 'cm', 'dt']
disc_value = test_list # list of metrics for each discretization after preparation
test_stat = []
p_value = []
disc_compare = []
# Create dictionary store discretization and series of bias
disc = {}
for key in disc_key:
    for value in disc_value:
        disc[key] = value
        disc_value.remove(value)
        break  

In [36]:
# Create loop for Wilcoxon test (two sided)
for i in disc_key:
    for j in disc_key:
        if i != j:
            disc_compare.append(f'{i} vs {j}')
            # print(f'{i} vs {j}')
            # Compute difference list
            diff_list = []
            for m in range(0, len(disc[i])):
                diff = disc[i][m] - disc[j][m]
                diff_list.append(diff)
                                
            if all(item == 0 for item in diff_list) == False: # if the diff list does not contain all 0
                test_stat.append(stats.wilcoxon(diff_list, alternative = 'greater').statistic)
                p_value.append(stats.wilcoxon(diff_list, alternative = 'greater').pvalue)
            else: # if the diff list contain only0, cannot do Wilcoxon test
                error = 'N/A'
                test_stat.append(error)
                p_value.append(error)


In [37]:
# Convert 3 lists to dataframe
disc_compare = pd.DataFrame(disc_compare, columns=['disc_compare'])
test_stat = pd.DataFrame(test_stat, columns=['wtest_stat'])
p_value = pd.DataFrame(p_value, columns=['p_value'])

In [38]:
# Result table
wt_result = pd.concat([disc_compare, test_stat, p_value], axis = 1)
# Drop duplicate
wt_result.drop_duplicates(subset=['p_value'], inplace=True)

In [39]:
wt_result

,disc_compare,wtest_stat,p_value
0,ewd vs efd,63.0,0.031982
1,ewd vs ffd,30.0,0.765137
2,ewd vs cm,45.0,0.338623
4,efd vs ewd,15.0,0.973877
5,efd vs ffd,27.0,0.830322
6,efd vs cm,38.0,0.545166
7,efd vs dt,42.0,0.425049
8,ffd vs ewd,48.0,0.259277
9,ffd vs efd,51.0,0.190186
10,ffd vs cm,52.0,0.169678


In [40]:
# Exporting result

model = 'KNN-Hamming'
metric = 'bias'
test = 'one_sided'

filename = f"wilcoxon_{metric}_{model}-{test}.csv"
wt_result['model'] = model
wt_result['metric'] = metric

wt_result.to_csv(filename, index = False)

## 1.6 bias, only KNN


In [41]:
# Preparation: Prepare list of metrics for each discretization (test_list)
# Step 1: Obtain bias for each discretization, convert into numeric, string values will be return as NaN
# Filter for Knn-VDM
# Obtain bias for each discretization, convert into numeric, string values will be return as NaN
ewd_acc_knn = pd.to_numeric(data[(data['disc']=="EWD") & ((data['models']=="Knn-Hamming") | (data['models']=="Knn-VDM"))]['bias'],errors='coerce').tolist()
efd_acc_knn = pd.to_numeric(data[(data['disc']=="EFD") & ((data['models']=="Knn-Hamming") | (data['models']=="Knn-VDM"))]['bias'],errors='coerce').tolist()
ffd_acc_knn = pd.to_numeric(data[(data['disc']=="FFD") & ((data['models']=="Knn-Hamming") | (data['models']=="Knn-VDM"))]['bias'],errors='coerce').tolist()
cm_acc_knn = pd.to_numeric(data[((data['disc']=="ChiMerge") | (data['disc'] == "ChiMerge-SB")) & ((data['models']=="Knn-Hamming") | (data['models']=="Knn-VDM"))]['bias'],errors='coerce').tolist()
dt_acc_knn = pd.to_numeric(data[(data['disc']=="ChiMerge") & ((data['models']=="Knn-Hamming") | (data['models']=="Knn-VDM"))]['bias'],errors='coerce').tolist()

# Check number of metrics available for each discretizer
print(len(ewd_acc_knn))
print(len(efd_acc_knn))
print(len(ffd_acc_knn))
print(len(cm_acc_knn))
print(len(dt_acc_knn))

# Step 2: filter numeric values
raw_list = [ewd_acc_knn,efd_acc_knn, ffd_acc_knn, cm_acc_knn, dt_acc_knn]
num_list = [] # filter numeric values only
for metric in raw_list:
    metric_new = [x for x in metric if (math.isnan(x) == False)] # using list comprehension
    num_list.append(metric_new)
    
# Step 3: random sample
# For discretization methods with different value, randomly select so that the final sample size are equal
# Reference: https://docs.python.org/3/library/random.html
k = min(len(metric) for metric in num_list)
test_list = []
for metric in num_list:
    random.seed(20)
    if len(metric) > k:
        metric = random.sample(metric, k=k)
    else: metric = metric
    #print(metric)
    test_list.append(metric)

30
30
40
40
20


In [42]:
# Initialiation
disc_key = ['ewd', 'efd', 'ffd', 'cm', 'dt']
disc_value = test_list # list of metrics for each discretization after preparation
test_stat = []
p_value = []
disc_compare = []
# Create dictionary store discretization and series of bias
disc = {}
for key in disc_key:
    for value in disc_value:
        disc[key] = value
        disc_value.remove(value)
        break  

In [43]:
# Create loop for Wilcoxon test (two sided)
for i in disc_key:
    for j in disc_key:
        if i != j:
            disc_compare.append(f'{i} vs {j}')
            # print(f'{i} vs {j}')
            # Compute difference list
            diff_list = []
            for m in range(0, len(disc[i])):
                diff = disc[i][m] - disc[j][m]
#                 print(disc[i][m])
#                 print(disc[j][m])
#                 print('diff = ', diff)
#                 print('-------------')
                diff_list.append(diff)
                # print(diff_list)
                
            if all(item == 0 for item in diff_list) == False: # if the diff list does not contain all 0
                test_stat.append(stats.wilcoxon(diff_list, alternative = 'greater').statistic)
                p_value.append(stats.wilcoxon(diff_list, alternative = 'greater').pvalue)
            else: # if the diff list contain only0, cannot do Wilcoxon test
                error = 'N/A'
                test_stat.append(error)
                p_value.append(error)
# print(test_stat)
# print(p_value)


/Users/anhtth/opt/anaconda3/lib/python3.9/site-packages/scipy/stats/morestats.py:3141: UserWarning: Exact p-value calculation does not work if there are ties. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "


In [44]:
# Convert 3 lists to dataframe
disc_compare = pd.DataFrame(disc_compare, columns=['disc_compare'])
test_stat = pd.DataFrame(test_stat, columns=['wtest_stat'])
p_value = pd.DataFrame(p_value, columns=['p_value'])

In [45]:
# Result table
wt_result = pd.concat([disc_compare, test_stat, p_value], axis = 1)
# Drop duplicate
wt_result.drop_duplicates(subset=['p_value'], inplace=True)

In [46]:
wt_result

,disc_compare,wtest_stat,p_value
0,ewd vs efd,94.0,0.215340
1,ewd vs ffd,64.5,0.726166
2,ewd vs cm,75.5,0.536743
3,ewd vs dt,95.5,0.201881
4,efd vs ewd,59.0,0.798119
5,efd vs ffd,38.5,0.968086
6,efd vs cm,56.0,0.835526
7,efd vs dt,82.0,0.408791
8,ffd vs ewd,88.5,0.305611
9,ffd vs efd,114.5,0.039841


In [47]:
# Exporting result

model = 'KNN'
metric = 'bias'
test = 'one_sided'

filename = f"wilcoxon_{metric}_{model}-{test}.csv"
wt_result['model'] = model
wt_result['metric'] = metric

wt_result.to_csv(filename, index = False)